In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scMVP.dataset import LoadData,GeneExpressionDataset, CellMeasurement
from scMVP.models import VAE_Attention, Multi_VAE_Attention, VAE_Peak_SelfAttention
from scMVP.inference import UnsupervisedTrainer
from scMVP.inference import MultiPosterior, MultiTrainer
import torch

import scanpy as sc
import anndata

import scipy.io as sp_io
from scipy.sparse import csr_matrix, issparse

[2021-11-23 10:13:25,778] INFO - scMVP._settings | Added StreamHandler with custom formatter to 'scMVP' logger.
/home/shaliu_fu/miniconda2/envs/scmvp/lib/python3.7/site-packages/scikit_learn-0.22.2-py3.7-linux-x86_64.egg/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [12]:
torch.cuda.is_available()
torch.set_num_threads(20)


In [3]:
input_path = "./10x_lymph_node/"
output_path = "./10x_lymph_node/scMVP_output"

lymph_node_dataset = {
                "gene_names": '10x_lymph_node_scale_gene.txt',
                "gene_expression": '10x_lymph_node_rna_normalize_count.mtx',
                "gene_barcodes": '10x_lymph_node_cell_barcode.txt',
                "atac_names": '10x_lymph_node_peak.txt',
                "atac_expression": '10x_lymph_node_atac_normalize_count.mtx',
                "atac_barcodes": '10x_lymph_node_cell_barcode.txt'
                }

dataset = LoadData(dataset=lymph_node_dataset,data_path=input_path,
                       dense=False,gzipped=False, atac_threshold=0.001,
                       cell_threshold=1)

[2021-11-23 10:20:39,072] INFO - scMVP.dataset.scMVP_dataloader | Preprocessing joint profiling dataset.
[2021-11-23 10:21:20,840] INFO - scMVP.dataset.scMVP_dataloader | Finished preprocessing dataset
[2021-11-23 10:21:20,927] INFO - scMVP.dataset.dataset | Remapping batch_indices to [0,N]
[2021-11-23 10:21:20,928] INFO - scMVP.dataset.dataset | Remapping labels to [0,N]
[2021-11-23 10:22:48,105] INFO - scMVP.dataset.dataset | Computing the library size for the new data
[2021-11-23 10:22:48,150] INFO - scMVP.dataset.dataset | Downsampled from 7039 to 7039 cells


In [10]:
# atac dataloader
atac_dataset = GeneExpressionDataset()
cell_attributes_dict = {
    "barcodes": dataset.barcodes
    }
atac_dataset.populate_from_data(
    X=dataset.atac_expression, # notice the normalization
    batch_indices=None,
    gene_names=dataset.atac_names,
    cell_attributes_dict=cell_attributes_dict,
    Ys=[],
)
rna_dataset = GeneExpressionDataset()
Ys = []
measurement = CellMeasurement(
        name="atac_expression",
        data=atac_dataset.X,
        columns_attr_name="atac_names",
        columns=atac_dataset.gene_names,
    )
Ys.append(measurement)
cell_attributes_dict = {
    "barcodes": dataset.barcodes
    }
rna_dataset.populate_from_data(
    X=dataset.X,
    batch_indices=None,
    gene_names=dataset.gene_names,
    cell_attributes_dict=cell_attributes_dict,
    Ys=Ys,
)

[2021-11-23 10:26:31,524] INFO - scMVP.dataset.dataset | Remapping batch_indices to [0,N]
[2021-11-23 10:26:31,526] INFO - scMVP.dataset.dataset | Remapping labels to [0,N]
[2021-11-23 10:26:32,349] INFO - scMVP.dataset.dataset | Remapping batch_indices to [0,N]
[2021-11-23 10:26:32,350] INFO - scMVP.dataset.dataset | Remapping labels to [0,N]


In [11]:
# 进行预训练

# model para
n_epochs = 15
lr = 5e-3
use_batches = False
use_cuda = True
n_centroids = 15
n_alfa = 1.0

# ATAC peak embedding
pre_atac_vae = VAE_Peak_SelfAttention(atac_dataset.nb_genes, n_latent=20,n_batch=0, n_layers=1, log_variational=True, reconstruction_loss="nb")
pre_atac_trainer = UnsupervisedTrainer(
    pre_atac_vae,
    atac_dataset,
    train_size=0.9,
    use_cuda=use_cuda,
    frequency=5,
)
is_test_pragram = False

if os.path.isfile('%s/pre_atac_trainerk.pkl' % output_path):
    pre_atac_trainer.model.load_state_dict(torch.load('%s/pre_atac_trainer.pkl' % output_path))
    pre_atac_trainer.model.eval()
else:
    pre_atac_trainer.train(n_epochs=n_epochs, lr=lr)
    torch.save(pre_atac_trainer.model.state_dict(), '%s/pre_atac_trainer.pkl' % output_path)
    pre_atac_trainer.model.eval()

training:   0%|          | 0/15 [00:00<?, ?it/s]reconst_loss=9125.333008, kl_divergence=0.088799
tensor(9126.3330, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=9254.258789, kl_divergence=0.420087
tensor(9255.2588, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8314.435547, kl_divergence=1.114696
tensor(8315.4355, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=9850.153320, kl_divergence=2.569014
tensor(9851.1533, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=9102.714844, kl_divergence=6.883841
tensor(9103.7148, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8570.944336, kl_divergence=16.938183
tensor(8571.9443, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=9065.055664, kl_divergence=37.709003
tensor(9066.0557, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=9515.861328, kl_divergence=63.096493
tensor(9516.8613, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8638.613281, kl_divergence=65.691856
tensor(8639.6133, device='cuda:0', 

reconst_loss=7503.891113, kl_divergence=62.380787
tensor(7504.8911, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7432.861328, kl_divergence=63.501621
tensor(7433.8613, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7322.632324, kl_divergence=63.107399
tensor(7323.6323, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8581.403320, kl_divergence=64.828659
tensor(8582.4033, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7171.101562, kl_divergence=61.345520
tensor(7172.1016, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8265.664062, kl_divergence=60.925644
tensor(8266.6641, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6788.920410, kl_divergence=61.713516
tensor(6789.9199, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6804.696777, kl_divergence=63.772766
tensor(6805.6968, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7492.392090, kl_divergence=63.288494
tensor(7493.3921, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7032.8

reconst_loss=7099.422363, kl_divergence=56.246025
tensor(7100.5625, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6946.735352, kl_divergence=56.739662
tensor(6947.8770, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7762.263184, kl_divergence=57.017845
tensor(7763.4062, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7834.073242, kl_divergence=54.276646
tensor(7835.2090, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8470.613281, kl_divergence=53.376984
tensor(8471.7461, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7830.607910, kl_divergence=51.728401
tensor(7831.7373, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8073.495605, kl_divergence=51.186691
tensor(8074.6230, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7806.469727, kl_divergence=52.307518
tensor(7807.5996, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8787.669922, kl_divergence=53.337708
tensor(8788.8047, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8252.5

reconst_loss=7700.290039, kl_divergence=56.522793
tensor(7701.5732, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7488.291016, kl_divergence=56.591419
tensor(7489.5742, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6995.848633, kl_divergence=56.945526
tensor(6997.1338, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6205.057617, kl_divergence=55.650185
tensor(6206.3354, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7618.054199, kl_divergence=54.632168
tensor(7619.3267, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6987.916504, kl_divergence=54.523010
tensor(6989.1890, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7872.727539, kl_divergence=54.084591
tensor(7873.9985, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7522.890625, kl_divergence=53.704330
tensor(7524.1582, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7570.966797, kl_divergence=53.063278
tensor(7572.2324, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6851.3

reconst_loss=7468.663086, kl_divergence=57.513840
tensor(7470.0942, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7772.580078, kl_divergence=57.963665
tensor(7774.0146, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6607.936035, kl_divergence=57.007160
tensor(6609.3633, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6699.752441, kl_divergence=57.919670
tensor(6701.1865, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7174.629883, kl_divergence=58.674229
tensor(7176.0698, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8105.976074, kl_divergence=61.306488
tensor(8107.4365, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6640.676758, kl_divergence=60.300327
tensor(6642.1289, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8279.327148, kl_divergence=59.776009
tensor(8280.7754, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8124.650391, kl_divergence=60.153618
tensor(8126.1011, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7226.9

...

reconst_loss=6954.800293, kl_divergence=67.027039
tensor(6957.8110, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7693.083984, kl_divergence=66.547638
tensor(7696.0806, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7496.486816, kl_divergence=65.711578
tensor(7499.4580, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7602.765625, kl_divergence=64.774803
tensor(7605.7090, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7588.247070, kl_divergence=64.057800
tensor(7591.1680, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6776.112305, kl_divergence=64.342339
tensor(6779.0430, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6889.112305, kl_divergence=62.289360
tensor(6891.9805, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7967.641602, kl_divergence=61.976830
tensor(7970.5010, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7133.604492, kl_divergence=60.517677
tensor(7136.4204, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7159.4

reconst_loss=7359.721191, kl_divergence=63.896454
tensor(7362.7979, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8170.255371, kl_divergence=63.788914
tensor(8173.3291, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6592.618164, kl_divergence=62.507854
tensor(6595.6504, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6760.808594, kl_divergence=63.012611
tensor(6763.8564, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8320.501953, kl_divergence=62.357727
tensor(8323.5273, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8135.930664, kl_divergence=63.111504
tensor(8138.9819, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7814.694336, kl_divergence=62.670647
tensor(7817.7305, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6043.122070, kl_divergence=62.060371
tensor(6046.1387, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7377.748047, kl_divergence=64.649200
tensor(7380.8491, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6649.4

reconst_loss=8114.720215, kl_divergence=65.332687
tensor(8118.0063, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7050.853027, kl_divergence=65.142593
tensor(7054.1328, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6871.723633, kl_divergence=67.401260
tensor(6875.0830, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6543.266113, kl_divergence=67.379013
tensor(6546.6240, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=6844.592773, kl_divergence=66.743065
tensor(6847.9287, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7366.905762, kl_divergence=67.190140
tensor(7370.2568, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7649.646484, kl_divergence=66.991989
tensor(7652.9912, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7068.676758, kl_divergence=66.879311
tensor(7072.0176, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=7503.826172, kl_divergence=66.883377
tensor(7507.1670, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=8334.4

reconst_loss=8666.126953, kl_divergence=60.213333
reconst_loss=7788.877441, kl_divergence=59.433041
reconst_loss=7367.659180, kl_divergence=60.113060
reconst_loss=7381.947266, kl_divergence=59.518669
reconst_loss=7884.723633, kl_divergence=58.959480
reconst_loss=6783.906250, kl_divergence=59.966465
reconst_loss=6728.690430, kl_divergence=58.821411
reconst_loss=7360.670898, kl_divergence=59.344955
reconst_loss=7049.729004, kl_divergence=58.971390
reconst_loss=7120.522949, kl_divergence=60.104393
reconst_loss=7059.621582, kl_divergence=60.960823
reconst_loss=6900.122559, kl_divergence=59.350082
reconst_loss=7694.797363, kl_divergence=59.727440
reconst_loss=7664.945312, kl_divergence=59.333626
reconst_loss=7927.193359, kl_divergence=60.638489
reconst_loss=7352.181641, kl_divergence=59.952328
reconst_loss=7376.677734, kl_divergence=60.264072
reconst_loss=7250.594727, kl_divergence=61.569405
reconst_loss=7037.785156, kl_divergence=60.288170
reconst_loss=7238.524902, kl_divergence=59.409405


reconst_loss=6663.971680, kl_divergence=58.759277
reconst_loss=8322.383789, kl_divergence=60.952389
reconst_loss=6816.692383, kl_divergence=58.955139
reconst_loss=7132.244141, kl_divergence=59.328072
reconst_loss=7848.956055, kl_divergence=59.670311
reconst_loss=7804.445312, kl_divergence=59.842842
reconst_loss=6927.117676, kl_divergence=59.617317
reconst_loss=6685.281250, kl_divergence=58.133545
reconst_loss=7110.363281, kl_divergence=60.212921
reconst_loss=8880.710938, kl_divergence=59.623146
reconst_loss=7321.100586, kl_divergence=59.124451
reconst_loss=7755.806641, kl_divergence=59.271194
reconst_loss=7150.927734, kl_divergence=60.584282
reconst_loss=6960.537109, kl_divergence=59.817207
reconst_loss=7465.416016, kl_divergence=60.564529
reconst_loss=8460.965820, kl_divergence=59.960686
reconst_loss=6072.417969, kl_divergence=58.583290
reconst_loss=7182.586914, kl_divergence=58.631878
reconst_loss=7882.950195, kl_divergence=59.910378
reconst_loss=6863.925293, kl_divergence=60.236115


In [13]:
# ATAC pretrainer_posterior:
full = pre_atac_trainer.create_posterior(pre_atac_trainer.model, atac_dataset, indices=np.arange(len(atac_dataset)))
latent, batch_indices, labels = full.sequential().get_latent()
batch_indices = batch_indices.ravel()
prior_adata = anndata.AnnData(X=atac_dataset.X)
prior_adata.obsm["X_multi_vi"] = latent
prior_adata.obs['cell_type'] = torch.tensor(labels.reshape(-1,1))
sc.pp.neighbors(prior_adata, use_rep="X_multi_vi", n_neighbors=30)
sc.tl.umap(prior_adata, min_dist=0.3)
#matplotlib.use('TkAgg')
#fig, ax = plt.subplots(figsize=(7, 6))
#sc.pl.umap(prior_adata, color=["cell_type"])
#plt.show()
sc.tl.louvain(prior_adata)
sc.pl.umap(prior_adata, color=['louvain'])
plt.show()
# save data as csv file
df = pd.DataFrame(data=prior_adata.obsm["X_umap"],  columns=["umap_dim1","umap_dim2"] , index=atac_dataset.barcodes )
df.insert(0,"labels",prior_adata.obs['louvain'].values)
df.to_csv(os.path.join(output_path,"scvi_atac_umap.csv"))

df = pd.DataFrame(data=prior_adata.obsm["X_multi_vi"],  index=atac_dataset.barcodes)
df.to_csv(os.path.join(output_path,"scvi_latent_atac_imputation.csv"))
imputed_values = full.sequential().imputation()
df = pd.DataFrame(data=imputed_values.T, columns=atac_dataset.barcodes, index=atac_dataset.gene_names)

In [14]:
# RNA embedding
pre_vae = VAE_Attention(rna_dataset.nb_genes, n_latent=20,n_batch=0, n_layers=1, log_variational=True, reconstruction_loss="nb")
pre_trainer = UnsupervisedTrainer(
    pre_vae,
    rna_dataset,
    train_size=0.9,
    use_cuda=use_cuda,
    frequency=5,
)
is_test_pragram = False
if is_test_pragram:
    pre_trainer.train(n_epochs=n_epochs, lr=lr)
    torch.save(pre_trainer.model.state_dict(), '%s/pre_trainer.pkl' % output_path)

if os.path.isfile('%s/pre_trainer.pkl' % output_path):
    pre_trainer.model.load_state_dict(torch.load('%s/pre_trainer.pkl' % output_path))
else:
    pre_trainer.train(n_epochs=n_epochs, lr=lr)
    torch.save(pre_trainer.model.state_dict(), '%s/pre_trainer.pkl' % output_path)
    pre_trainer.model.eval()

# RNA pretrainer_posterior:
full = pre_trainer.create_posterior(pre_trainer.model, rna_dataset, indices=np.arange(len(rna_dataset)))
latent, batch_indices, labels = full.sequential().get_latent()
batch_indices = batch_indices.ravel()
imputed_values = full.sequential().imputation()

df = pd.DataFrame(data=imputed_values.T, columns=rna_dataset.barcodes, index=rna_dataset.gene_names)
#df.to_csv(os.path.join(save_path,"gene_scvi_imputation_210324_2.csv"))
# visulization
prior_adata = anndata.AnnData(X=rna_dataset.X)
prior_adata.obsm["X_multi_vi"] = latent
prior_adata.obs['cell_type'] = torch.tensor(labels.reshape(-1,1))
sc.pp.neighbors(prior_adata, use_rep="X_multi_vi", n_neighbors=30)
sc.tl.umap(prior_adata, min_dist=0.3)
#matplotlib.use('TkAgg')
#fig, ax = plt.subplots(figsize=(7, 6))
#sc.pl.umap(prior_adata, color=["cell_type"], ax=ax, show=show_plot)
#plt.show()
sc.tl.louvain(prior_adata)
sc.pl.umap(prior_adata, color=['louvain'])
plt.show()

# save data as csv file
df = pd.DataFrame(data=prior_adata.obsm["X_umap"],  columns=["umap_dim1","umap_dim2"] , index=rna_dataset.barcodes )
df.insert(0,"labels",prior_adata.obs['louvain'].values)
df.to_csv(os.path.join(output_path,"scvi_umap.csv"))

df = pd.DataFrame(data=prior_adata.obsm["X_multi_vi"],  index=rna_dataset.barcodes)
df.to_csv(os.path.join(output_path,"scvi_latent_imputation.csv"))

training:   0%|          | 0/15 [00:00<?, ?it/s]reconst_loss=1199.452026, kl_divergence=0.672067
tensor(1200.4520, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=1148.259644, kl_divergence=0.903855
tensor(1149.2596, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=1070.145752, kl_divergence=1.356137
tensor(1071.1458, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=951.737305, kl_divergence=2.047019
tensor(952.7373, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=926.873535, kl_divergence=3.537813
tensor(927.8735, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=831.660217, kl_divergence=5.588002
tensor(832.6602, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=786.705566, kl_divergence=8.687210
tensor(787.7056, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=779.406311, kl_divergence=14.239708
tensor(780.4063, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=749.899292, kl_divergence=21.569992
tensor(750.8993, device='cuda:0', grad_fn=<DivBa

reconst_loss=634.147766, kl_divergence=76.453796
tensor(635.1478, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=611.898682, kl_divergence=77.181694
tensor(612.8987, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=630.787109, kl_divergence=77.247772
tensor(631.7871, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=635.204590, kl_divergence=75.637360
tensor(636.2046, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=630.183838, kl_divergence=77.242615
tensor(631.1838, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=646.910461, kl_divergence=78.201492
tensor(647.9105, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=616.549927, kl_divergence=77.008278
tensor(617.5499, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=653.387878, kl_divergence=80.244499
tensor(654.3879, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=619.033447, kl_divergence=78.063835
tensor(620.0334, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=632.760010, kl_divergenc

reconst_loss=614.494995, kl_divergence=89.900284
tensor(615.7197, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=585.941895, kl_divergence=91.582039
tensor(587.1709, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=645.009888, kl_divergence=92.352753
tensor(646.2408, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=616.139221, kl_divergence=92.155319
tensor(617.3696, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=617.932861, kl_divergence=91.269798
tensor(619.1610, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=605.965210, kl_divergence=92.021103
tensor(607.1953, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=628.863647, kl_divergence=90.571808
tensor(630.0901, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=626.250366, kl_divergence=92.466263
tensor(627.4816, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=594.143921, kl_divergence=92.974945
tensor(595.3763, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=630.743225, kl_divergenc

reconst_loss=587.886963, kl_divergence=93.481651
tensor(589.3544, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=611.476868, kl_divergence=94.320496
tensor(612.9485, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=608.935913, kl_divergence=96.641953
tensor(610.4191, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=612.629700, kl_divergence=94.716125
tensor(614.1033, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=620.420105, kl_divergence=93.199844
tensor(621.8861, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=624.156799, kl_divergence=94.526382
tensor(625.6295, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=607.006348, kl_divergence=94.011414
tensor(608.4764, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=631.153687, kl_divergence=94.537033
tensor(632.6264, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=625.370483, kl_divergence=95.223854
tensor(626.8466, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=611.212158, kl_divergenc

reconst_loss=616.108154, kl_divergence=92.583191
tensor(617.8026, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=606.214844, kl_divergence=94.723633
tensor(607.9253, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=589.672363, kl_divergence=94.448616
tensor(591.3807, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=624.279602, kl_divergence=94.058838
tensor(625.9850, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=601.731934, kl_divergence=93.757889
tensor(603.4351, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=616.782471, kl_divergence=93.429306
tensor(618.4832, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=569.736511, kl_divergence=92.982529
tensor(571.4338, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=594.019531, kl_divergence=93.302620
tensor(595.7192, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=615.009460, kl_divergence=92.403770
tensor(616.7025, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=587.820068, kl_divergenc

...

reconst_loss=600.691650, kl_divergence=67.624939
tensor(603.7205, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=610.960327, kl_divergence=67.841095
tensor(613.9956, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=595.928101, kl_divergence=67.851089
tensor(598.9637, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=575.238647, kl_divergence=66.814590
tensor(578.2430, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=589.493408, kl_divergence=67.100235
tensor(592.5065, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=589.350952, kl_divergence=67.983536
tensor(592.3905, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=583.242554, kl_divergence=67.006233
tensor(586.2527, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=591.763794, kl_divergence=67.221558
tensor(594.7805, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=569.906311, kl_divergence=67.828278
tensor(572.9412, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=591.027832, kl_divergenc

reconst_loss=575.686523, kl_divergence=68.277657
tensor(578.9056, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=588.625122, kl_divergence=66.716614
tensor(591.7935, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=586.984131, kl_divergence=66.586517
tensor(590.1482, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=586.342163, kl_divergence=67.237930
tensor(589.5275, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=589.693726, kl_divergence=66.863983
tensor(592.8668, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=592.592163, kl_divergence=67.610153
tensor(595.7894, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=590.546204, kl_divergence=67.877396
tensor(593.7522, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=574.587769, kl_divergence=68.095505
tensor(577.8009, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=578.404419, kl_divergence=65.455345
tensor(581.5317, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=576.553955, kl_divergenc

reconst_loss=586.361694, kl_divergence=66.929184
tensor(589.7042, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=548.816101, kl_divergence=66.401993
tensor(552.1401, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=560.524597, kl_divergence=67.003502
tensor(563.8697, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=577.932495, kl_divergence=68.262009
tensor(581.3217, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=596.788940, kl_divergence=66.493393
tensor(600.1162, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=572.617493, kl_divergence=66.385956
tensor(575.9410, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=574.688721, kl_divergence=66.482025
tensor(578.0156, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=589.204956, kl_divergence=67.994194
tensor(592.5847, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=575.724304, kl_divergence=66.810455
tensor(579.0626, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=586.235107, kl_divergenc

reconst_loss=585.157715, kl_divergence=65.464127
tensor(588.4490, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=578.559143, kl_divergence=63.421627
tensor(581.7790, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=576.599182, kl_divergence=64.027641
tensor(579.8402, device='cuda:0', grad_fn=<DivBackward0>)
reconst_loss=576.117371, kl_divergence=63.019077
reconst_loss=583.682007, kl_divergence=61.687061
reconst_loss=556.178589, kl_divergence=61.521656
reconst_loss=582.863586, kl_divergence=62.913456
reconst_loss=574.858154, kl_divergence=61.143951
reconst_loss=547.859375, kl_divergence=61.801983
reconst_loss=566.607483, kl_divergence=62.497200
reconst_loss=573.582947, kl_divergence=62.703506
reconst_loss=573.871216, kl_divergence=63.800987
reconst_loss=571.589111, kl_divergence=60.553680
reconst_loss=584.776978, kl_divergence=64.359840
reconst_loss=542.604614, kl_divergence=63.331345
reconst_loss=544.170959, kl_divergence=61.628616
reconst_loss=579.333557, kl_divergence=60.

reconst_loss=573.510376, kl_divergence=62.868362
reconst_loss=554.262085, kl_divergence=60.992725
reconst_loss=568.926025, kl_divergence=62.847252
reconst_loss=575.838013, kl_divergence=60.571449
reconst_loss=568.200439, kl_divergence=61.766697
reconst_loss=559.683838, kl_divergence=61.270321
reconst_loss=569.552795, kl_divergence=61.715530
reconst_loss=565.359619, kl_divergence=64.090630
reconst_loss=569.586304, kl_divergence=61.552177
reconst_loss=572.751343, kl_divergence=62.817585
reconst_loss=568.283569, kl_divergence=60.586136
reconst_loss=567.166382, kl_divergence=61.812176
reconst_loss=575.046326, kl_divergence=61.689789
reconst_loss=563.541260, kl_divergence=60.253616
reconst_loss=540.712769, kl_divergence=60.610336
reconst_loss=578.133667, kl_divergence=62.462742
reconst_loss=565.893433, kl_divergence=62.812019
reconst_loss=579.180725, kl_divergence=62.269154
reconst_loss=594.066040, kl_divergence=60.692383
reconst_loss=571.924011, kl_divergence=59.930397
reconst_loss=561.478

In [15]:
n_centroids=len(np.unique(prior_adata.obs['louvain'].tolist()))

# joint RNA and ATAC embedding
multi_vae = Multi_VAE_Attention(rna_dataset.nb_genes, len(rna_dataset.atac_names), n_batch=0, n_latent=20, n_centroids=n_centroids, n_alfa = n_alfa, mode="mm-vae") # should provide ATAC num, alfa, mode and loss type
trainer = MultiTrainer(
    multi_vae,
    rna_dataset,
    train_size=0.9,
    use_cuda=use_cuda,
    frequency=5,
)


if os.path.isfile('%s/multi_vae.pkl' % output_path):
    trainer.model.load_state_dict(torch.load('%s/multi_vae.pkl' % output_path))
    trainer.model.eval()
else:
    pre_trainer = UnsupervisedTrainer(
        pre_vae,
        rna_dataset,
        train_size=0.9,
        use_cuda=use_cuda,
        frequency=5,
    )
    pre_trainer.model.load_state_dict(torch.load('%s/pre_trainer.pkl' % output_path))

    pre_atac_trainer = UnsupervisedTrainer(
        pre_atac_vae,
        atac_dataset,
        train_size=0.9,
        use_cuda=use_cuda,
        frequency=5,
    )
    pre_atac_trainer.model.load_state_dict(torch.load('%s/pre_atac_trainer.pkl' % output_path))

    n_centroids=len(np.unique(prior_adata.obs['louvain'].tolist()))

    # joint RNA and ATAC embedding
    trainer.model.init_gmm_params_with_louvain(latent,np.array(prior_adata.obs['louvain'].tolist()).astype(int))

    trainer.model.RNA_encoder.load_state_dict(pre_trainer.model.z_encoder.state_dict())
    for param in trainer.model.RNA_encoder.parameters():
        param.requires_grad = True
    trainer.model.ATAC_encoder.load_state_dict(pre_atac_trainer.model.z_encoder.state_dict())
    for param in trainer.model.ATAC_encoder.parameters():
        param.requires_grad = True
    trainer.train(n_epochs=15, lr=lr)
    torch.save(trainer.model.state_dict(), '%s/multi_vae.pkl' % output_path)
    trainer.model.eval()

training:   0%|          | 0/15 [00:00<?, ?it/s]logpzc:-287.84307861328125, logqcx:-2.079441547393799
kld_qz_pz = 246.227661,kld_qz_rna = 5933.751953,kld_qz_atac = 18137.398438,kl_divergence = 271.020050,reconst_loss_rna = 3568.731445,        reconst_loss_atac = 857.273010, mu=0.116825, sigma=1.686523
reconst_loss = 4476.372070,kl_divergence_local = 271.020050,kl_weight = 1.000000,loss = 4747.392578
tensor(4747.3926, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-46.061073303222656, logqcx:-2.079441547393799
kld_qz_pz = 9.502743,kld_qz_rna = 4684.460449,kld_qz_atac = 987.074951,kl_divergence = 15.104197,reconst_loss_rna = 3362.215332,        reconst_loss_atac = 831.204224, mu=0.116400, sigma=1.690874
reconst_loss = 4243.787598,kl_divergence_local = 15.104197,kl_weight = 1.000000,loss = 4258.892090
tensor(4258.8921, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-43.807518005371094, logqcx:-2.079441547393799
kld_qz_pz = 9.222754,kld_qz_rna = 4216.265625,kld_qz_atac = 259.255676,kl_di

kld_qz_pz = 7.434896,kld_qz_rna = 1525.923828,kld_qz_atac = 376.645874,kl_divergence = 10.437126,reconst_loss_rna = 1986.721558,        reconst_loss_atac = 1012.398254, mu=0.106982, sigma=1.741041
reconst_loss = 3049.488037,kl_divergence_local = 10.437126,kl_weight = 1.000000,loss = 3059.925049
tensor(3059.9250, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-38.041839599609375, logqcx:-2.079437732696533
kld_qz_pz = 6.691964,kld_qz_rna = 1649.321777,kld_qz_atac = 228.614502,kl_divergence = 9.539717,reconst_loss_rna = 2143.903320,        reconst_loss_atac = 805.674133, mu=0.106771, sigma=1.742744
reconst_loss = 2999.945557,kl_divergence_local = 9.539717,kl_weight = 1.000000,loss = 3009.485352
tensor(3009.4854, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-37.459205627441406, logqcx:-2.0794291496276855
kld_qz_pz = 6.176533,kld_qz_rna = 1429.081543,kld_qz_atac = 220.295380,kl_divergence = 8.953671,reconst_loss_rna = 2055.962402,        reconst_loss_atac = 790.665710, mu=0.106596, sigm

logpzc:-35.75185775756836, logqcx:-2.0794224739074707
kld_qz_pz = 5.638467,kld_qz_rna = 769.015930,kld_qz_atac = 37.200214,kl_divergence = 8.675638,reconst_loss_rna = 1928.748779,        reconst_loss_atac = 754.667297, mu=0.107400, sigma=1.765991
reconst_loss = 2733.783936,kl_divergence_local = 8.675638,kl_weight = 1.000000,loss = 2742.459717
tensor(2742.4597, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-35.90188980102539, logqcx:-2.07942533493042
kld_qz_pz = 5.722612,kld_qz_rna = 772.666626,kld_qz_atac = 38.509476,kl_divergence = 8.737254,reconst_loss_rna = 2104.901855,        reconst_loss_atac = 811.464722, mu=0.107634, sigma=1.766704
reconst_loss = 2966.734375,kl_divergence_local = 8.737254,kl_weight = 1.000000,loss = 2975.471680
tensor(2975.4717, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-36.04742431640625, logqcx:-2.0794217586517334
kld_qz_pz = 5.898539,kld_qz_rna = 579.984192,kld_qz_atac = 38.344681,kl_divergence = 8.973586,reconst_loss_rna = 2019.355469,        reconst

logpzc:-37.00790786743164, logqcx:-2.079432487487793
kld_qz_pz = 5.903799,kld_qz_rna = 649.324097,kld_qz_atac = 37.605194,kl_divergence = 8.641756,reconst_loss_rna = 1970.557861,        reconst_loss_atac = 721.948792, mu=0.109919, sigma=1.778276
reconst_loss = 2742.874756,kl_divergence_local = 8.641756,kl_weight = 1.000000,loss = 2751.516357
tensor(2751.5164, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-37.4891357421875, logqcx:-2.079430103302002
kld_qz_pz = 6.282133,kld_qz_rna = 547.873657,kld_qz_atac = 38.325630,kl_divergence = 9.114414,reconst_loss_rna = 1988.956543,        reconst_loss_atac = 772.844971, mu=0.110182, sigma=1.778881
reconst_loss = 2812.169189,kl_divergence_local = 9.114414,kl_weight = 1.000000,loss = 2821.283936
tensor(2821.2839, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-37.618350982666016, logqcx:-2.0794315338134766
kld_qz_pz = 6.477922,kld_qz_rna = 635.335876,kld_qz_atac = 40.015675,kl_divergence = 9.389725,reconst_loss_rna = 1920.216797,        reconst

logpzc:-35.51699447631836, logqcx:-2.079420566558838
kld_qz_pz = 5.193516,kld_qz_rna = 493.896179,kld_qz_atac = 44.006599,kl_divergence = 7.959767,reconst_loss_rna = 2014.837402,        reconst_loss_atac = 806.537109, mu=0.113178, sigma=1.792104
reconst_loss = 2871.742188,kl_divergence_local = 7.959767,kl_weight = 1.000000,loss = 2879.702148
tensor(2879.7021, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-35.43891525268555, logqcx:-2.0794272422790527
kld_qz_pz = 5.109072,kld_qz_rna = 464.163269,kld_qz_atac = 44.440559,kl_divergence = 7.838649,reconst_loss_rna = 2068.504883,        reconst_loss_atac = 869.861450, mu=0.113183, sigma=1.792720
reconst_loss = 2988.734375,kl_divergence_local = 7.838649,kl_weight = 1.000000,loss = 2996.572998
tensor(2996.5730, device='cuda:0', grad_fn=<DivBackward0>)
logpzc:-35.2779541015625, logqcx:-2.079423427581787
kld_qz_pz = 5.022397,kld_qz_rna = 461.880280,kld_qz_atac = 43.499184,kl_divergence = 7.779515,reconst_loss_rna = 1985.383545,        reconst_

...

logpzc:-29.949512481689453, logqcx:-0.02022586017847061
kld_qz_pz = 0.132495,kld_qz_rna = 89.879166,kld_qz_atac = 90.830566,kl_divergence = 4.200336,reconst_loss_rna = 1731.656006,        reconst_loss_atac = 788.268127, mu=0.158373, sigma=1.799037
logpzc:-29.94100570678711, logqcx:-0.01882471889257431
kld_qz_pz = 0.125105,kld_qz_rna = 89.360214,kld_qz_atac = 91.011566,kl_divergence = 4.133474,reconst_loss_rna = 1783.476196,        reconst_loss_atac = 744.270020, mu=0.158373, sigma=1.799037
logpzc:-29.853931427001953, logqcx:-0.017533203586935997
kld_qz_pz = 0.038939,kld_qz_rna = 87.625832,kld_qz_atac = 90.980240,kl_divergence = 4.037556,reconst_loss_rna = 1869.212280,        reconst_loss_atac = 701.954956, mu=0.158373, sigma=1.799037
logpzc:-29.910743713378906, logqcx:-0.018733087927103043
kld_qz_pz = 0.094841,kld_qz_rna = 86.233887,kld_qz_atac = 91.045944,kl_divergence = 4.019294,reconst_loss_rna = 1735.894043,        reconst_loss_atac = 788.107666, mu=0.158373, sigma=1.799037
logpzc:

training: 100%|██████████| 15/15 [02:25<00:00,  9.72s/it]
logpzc:-29.954814910888672, logqcx:-0.01890757866203785
kld_qz_pz = 0.138804,kld_qz_rna = 91.727951,kld_qz_atac = 91.157631,kl_divergence = 4.138890,reconst_loss_rna = 1778.038086,        reconst_loss_atac = 757.010864, mu=0.158373, sigma=1.799037
logpzc:-29.903770446777344, logqcx:-0.018651606515049934
kld_qz_pz = 0.087988,kld_qz_rna = 89.859802,kld_qz_atac = 91.154488,kl_divergence = 4.124632,reconst_loss_rna = 1753.792969,        reconst_loss_atac = 653.684875, mu=0.158373, sigma=1.799037
logpzc:-29.945724487304688, logqcx:-0.018924634903669357
kld_qz_pz = 0.129733,kld_qz_rna = 91.424347,kld_qz_atac = 91.153198,kl_divergence = 4.098310,reconst_loss_rna = 1817.390503,        reconst_loss_atac = 781.542847, mu=0.158373, sigma=1.799037
logpzc:-30.024080276489258, logqcx:-0.02064727433025837
kld_qz_pz = 0.206606,kld_qz_rna = 91.736282,kld_qz_atac = 91.023361,kl_divergence = 4.215245,reconst_loss_rna = 1722.080078,        reconst_

logpzc:-29.906047821044922, logqcx:-0.018214909359812737
kld_qz_pz = 0.090415,kld_qz_rna = 90.040993,kld_qz_atac = 90.768814,kl_divergence = 4.043533,reconst_loss_rna = 1681.079346,        reconst_loss_atac = 645.459229, mu=0.158373, sigma=1.799037
logpzc:-29.966197967529297, logqcx:-0.019135046750307083
kld_qz_pz = 0.149889,kld_qz_rna = 89.457443,kld_qz_atac = 90.951416,kl_divergence = 4.133798,reconst_loss_rna = 1702.669189,        reconst_loss_atac = 735.464355, mu=0.158373, sigma=1.799037
logpzc:-29.889102935791016, logqcx:-0.017933830618858337
kld_qz_pz = 0.073768,kld_qz_rna = 89.576073,kld_qz_atac = 90.921173,kl_divergence = 4.016088,reconst_loss_rna = 1868.348755,        reconst_loss_atac = 873.966797, mu=0.158373, sigma=1.799037
logpzc:-29.90842056274414, logqcx:-0.018104188144207
kld_qz_pz = 0.093088,kld_qz_rna = 88.910950,kld_qz_atac = 91.026031,kl_divergence = 4.076343,reconst_loss_rna = 1764.947266,        reconst_loss_atac = 896.280151, mu=0.158373, sigma=1.799037
logpzc:-

kld_qz_pz = 0.128916,kld_qz_rna = 88.450668,kld_qz_atac = 91.456848,kl_divergence = 4.160156,reconst_loss_rna = 1767.443848,        reconst_loss_atac = 735.968018, mu=0.158373, sigma=1.799037
logpzc:-29.90085220336914, logqcx:-0.018050622195005417
kld_qz_pz = 0.085602,kld_qz_rna = 88.361717,kld_qz_atac = 91.111549,kl_divergence = 4.040233,reconst_loss_rna = 1777.401978,        reconst_loss_atac = 822.298462, mu=0.158373, sigma=1.799037
logpzc:-29.90607452392578, logqcx:-0.017721308395266533
kld_qz_pz = 0.090962,kld_qz_rna = 89.396996,kld_qz_atac = 91.189514,kl_divergence = 4.059210,reconst_loss_rna = 1700.548340,        reconst_loss_atac = 669.758545, mu=0.158373, sigma=1.799037
logpzc:-29.864269256591797, logqcx:-0.017320483922958374
kld_qz_pz = 0.049427,kld_qz_rna = 87.525787,kld_qz_atac = 90.966942,kl_divergence = 4.006605,reconst_loss_rna = 1808.491821,        reconst_loss_atac = 664.507935, mu=0.158373, sigma=1.799037
logpzc:-29.993831634521484, logqcx:-0.01985449157655239
kld_qz_

logpzc:-29.8114013671875, logqcx:-0.01652890257537365
kld_qz_pz = -0.002581,kld_qz_rna = 87.341721,kld_qz_atac = 91.255630,kl_divergence = 3.958299,reconst_loss_rna = 1792.736450,        reconst_loss_atac = 646.043091, mu=0.158373, sigma=1.799037
logpzc:-29.793384552001953, logqcx:-0.01623871549963951
kld_qz_pz = -0.020309,kld_qz_rna = 89.033455,kld_qz_atac = 91.187546,kl_divergence = 4.007509,reconst_loss_rna = 1811.413086,        reconst_loss_atac = 707.693726, mu=0.158373, sigma=1.799037
logpzc:-29.811634063720703, logqcx:-0.01636490784585476
kld_qz_pz = -0.002233,kld_qz_rna = 90.384445,kld_qz_atac = 91.355484,kl_divergence = 3.918237,reconst_loss_rna = 1785.967163,        reconst_loss_atac = 754.048767, mu=0.158373, sigma=1.799037
logpzc:-29.804462432861328, logqcx:-0.016383811831474304
kld_qz_pz = -0.009476,kld_qz_rna = 87.369164,kld_qz_atac = 91.055565,kl_divergence = 4.000754,reconst_loss_rna = 1766.723755,        reconst_loss_atac = 704.606262, mu=0.158373, sigma=1.799037
logpz

In [16]:
# 输出模型结果
# posterior
full = trainer.create_posterior(trainer.model, dataset, indices=np.arange(len(dataset)),type_class=MultiPosterior)
latent, latent_rna, latent_atac, cluster_gamma, cluster_index, batch_indices, labels = full.sequential().get_latent()
batch_indices = batch_indices.ravel()
imputed_values = full.sequential().imputation()
# visulization
prior_adata = anndata.AnnData(X=latent)
prior_adata.obsm["X_multi_vi"] = latent
prior_adata.obs['cell_type'] = torch.tensor(labels.reshape(-1,1))
sc.pp.neighbors(prior_adata, use_rep="X_multi_vi", n_neighbors=30)
sc.tl.umap(prior_adata, min_dist=0.3)
## matplotlib.use('TkAgg')
#fig, ax = plt.subplots(figsize=(7, 6))
#sc.pl.umap(prior_adata, color=["cell_type"], ax=ax, show=show_plot)
#plt.show()
sc.tl.louvain(prior_adata)
sc.pl.umap(prior_adata, color=['louvain'])
plt.show()

# save file
df = pd.DataFrame(data=prior_adata.obsm["X_multi_vi"],  index=rna_dataset.barcodes)
df.to_csv(os.path.join(output_path,"multivae_latent_imputation.csv"))

df = pd.DataFrame(data=prior_adata.obsm["X_umap"],  columns=["umap_dim1","umap_dim2"] , index=rna_dataset.barcodes)
df.insert(0,"louvain",prior_adata.obs['louvain'].values)
df.to_csv(os.path.join(output_path,"multivae_umap_louvain.csv"))

df = pd.DataFrame(data=imputed_values[1].T, columns=rna_dataset.barcodes, index=rna_dataset.atac_names)
#df.to_csv(os.path.join(output_path,"atac_multivae_imputation_softmax.csv"))

df = pd.DataFrame(data=imputed_values[0].T, columns=rna_dataset.barcodes, index=rna_dataset.gene_names)
#df.to_csv(os.path.join(output_path,"gene_multivae_imputation_softmax.csv"))